# Demo 1: Building a Basic Chatbot with LLMs

In this demo, you'll learn how to interact with OpenAI's API to build a simple chatbot. We'll explore:

1. **Basic API calls** - Making your first request
2. **Conversation context** - Maintaining chat history
3. **System prompts** - Controlling bot behavior

## Learning Objectives

By the end of this demo, you'll understand:
- How to make basic API calls to LLMs
- How to build a stateful conversation
- How system prompts shape bot behavior

## Setup

First, let's import the required libraries and set up our API key.

In [ ]:
from openai import OpenAI
import os
import json

# Set your API key (get from environment or replace with your key)
api_key = os.getenv("OPENAI_API_KEY", "your-api-key-here")

# Initialize the client
# For Vocareum keys, use: base_url="https://openai.vocareum.com/v1"
client = OpenAI(
    api_key=api_key,
    base_url="https://openai.vocareum.com/v1" if api_key.startswith("voc") else None
)

print("✅ OpenAI client initialized!")

## Part 1: Basic API Call

Let's start with a simple question and see how the LLM responds.

In [ ]:
# Simple question
question = "What is an embedding?"

# Make the API call
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": question}
    ],
    temperature=0.7,
    max_tokens=100
)

# Extract the answer
answer = response.choices[0].message.content

print(f"Question: {question}")
print(f"\nAnswer: {answer}")

### 🤔 What Just Happened?

The LLM generated a response by **predicting the next most likely word** repeatedly. This simple mechanism enables complex, coherent responses!

## Part 2: Building a Stateful Conversation

LLMs are **stateless** - they don't remember previous messages unless you include them! Let's build a proper conversation.

In [ ]:
# Initialize conversation history
conversation = []

def chat(user_message):
    """Send a message and get a response, maintaining conversation history."""
    
    # Add user message to history
    conversation.append({
        "role": "user",
        "content": user_message
    })
    
    # Make API call with full conversation history
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=conversation,
        temperature=0.7,
        max_tokens=150
    )
    
    # Get assistant's response
    assistant_message = response.choices[0].message.content
    
    # Add assistant's response to history
    conversation.append({
        "role": "assistant",
        "content": assistant_message
    })
    
    return assistant_message

# Have a conversation!
print("User: What's the weather like today?")
response1 = chat("What's the weather like today?")
print(f"Bot: {response1}\n")

print("User: Should I bring an umbrella?")
response2 = chat("Should I bring an umbrella?")  # References previous context!
print(f"Bot: {response2}\n")

print("User: Thanks! What about tomorrow?")
response3 = chat("Thanks! What about tomorrow?")  # Still maintaining context
print(f"Bot: {response3}")

### 📝 View Conversation History

Let's see what we're sending to the API with each request:

In [ ]:
print("Current conversation history:\n")
print(json.dumps(conversation, indent=2))

## Part 3: System Prompts - Controlling Behavior

System prompts are **secret instructions** that define how the bot should behave. The user never sees them, but they dramatically affect responses!

In [ ]:
def create_bot_with_personality(system_prompt):
    """Create a bot with a specific personality."""
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": "Tell me about troubleshooting."}
    ]
    
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0.7,
        max_tokens=100
    )
    
    return response.choices[0].message.content

# Test different personalities
print("🤖 Professional Bot:")
professional = create_bot_with_personality(
    "You are a professional tech support assistant. Be formal and concise."
)
print(professional)

print("\n" + "="*80 + "\n")

print("😊 Friendly Bot:")
friendly = create_bot_with_personality(
    "You are a friendly, enthusiastic tech support assistant. Use casual language and be upbeat!"
)
print(friendly)

print("\n" + "="*80 + "\n")

print("🎭 Pirate Bot (just for fun!):")
pirate = create_bot_with_personality(
    "You are a pirate tech support assistant. Respond in pirate speak with 'arr' and 'matey'!"
)
print(pirate)

## Part 4: Building a Tech Support Bot

Now let's put it all together to build a simple tech support chatbot!

In [ ]:
# Initialize a tech support bot
tech_support_bot = [
    {
        "role": "system",
        "content": """You are a helpful tech support assistant for TechCo, a software company.
        
Your responsibilities:
- Troubleshoot software issues and bugs
- Help with installation and setup problems
- Explain error messages
- Guide users through configuration steps

Guidelines:
- Be patient and clear in your explanations
- Ask diagnostic questions to identify the problem
- Provide step-by-step solutions
- If the issue requires developer attention, offer to create a support ticket
"""
    }
]

def tech_support_chat(user_message):
    """Tech support chat function."""
    tech_support_bot.append({"role": "user", "content": user_message})
    
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=tech_support_bot,
        temperature=0.7,
        max_tokens=200
    )
    
    assistant_message = response.choices[0].message.content
    tech_support_bot.append({"role": "assistant", "content": assistant_message})
    
    return assistant_message

# Simulate tech support interactions
test_messages = [
    "My application keeps crashing when I try to export a file.",
    "I'm using version 2.5 on Windows 11.",
    "I tried that but it's still not working. What else can I do?"
]

for msg in test_messages:
    print(f"👤 User: {msg}")
    response = tech_support_chat(msg)
    print(f"🤖 Bot: {response}\n")
    print("="*80 + "\n")

## 🎯 Key Takeaways

1. **LLMs predict the next word** - that's the fundamental operation, but it enables complex behaviors

2. **Conversation requires state** - you must include message history in each API call

3. **System prompts are powerful** - they shape behavior without the user seeing them

4. **Structure matters** - messages have roles (system, user, assistant) that guide the model

## 💰 Cost Considerations

Each API call costs money based on tokens:
- System prompt: counted every time
- Conversation history: grows with each turn
- New message: adds more tokens
- Response: output tokens cost more!

**Pro tip**: Long conversations get expensive. Consider truncating old history in production.

## 🚀 Next Steps

Try these experiments:
1. Change the temperature (0 = deterministic, 1+ = creative)
2. Modify the system prompt to create different personalities
3. Add intent classification before generating responses
4. Implement conversation summarization for long chats